In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [3]:
# Importing the dataset
data = pd.read_csv('diabetes.csv')
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [5]:
# Seperate the features and labels
features = data.iloc[:, :-1].values
labels = data.iloc[:, -1]

In [21]:
# One hot encoding the labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(labels).astype(float)

In [22]:
# Scaling/Normalizing the features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(features)

In [23]:
# Convert the np_array into torch.tensor
X = torch.from_numpy(X)
y = torch.from_numpy(y).unsqueeze(1)

In [25]:
print(X.shape, y.shape)

torch.Size([768, 7]) torch.Size([768, 1])


In [26]:
# Prepare custom dataset using torch.utils.data.Dataset
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [27]:
dataset = Dataset(X, y)

In [30]:
# Prepare dataloader
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=32,
                                     shuffle=True)

In [32]:
# Examine the batch size
print("Number of batches:", len(train_loader))
for (x, y) in train_loader:
    print("Shape of x:", x.shape)
    print("Shape of y:", y.shape)
    break

Number of batches: 24
Shape of x: torch.Size([32, 7])
Shape of y: torch.Size([32, 1])


In [52]:
# Defining the model
class Model(nn.Module):
    def __init__(self, num_of_features, num_of_output_classes):
        super().__init__()
        self.fc1 = nn.Linear(num_of_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, num_of_output_classes)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, X):
        out = self.fc1(X)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [53]:
# Building the model
model = Model(7, 1)

# Define the loss function
criterion = nn.BCELoss(reduction='mean')

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [58]:
# Training the network
epochs = 200
for epoch in range(epochs):
    for features, label in train_loader:
        # Forward propagation
        output = model(features.float())
        # Calculating loss
        loss = criterion(output.float(), label.float())
        # make gradient zero
        optimizer.zero_grad()
        # Backward propagation
        loss.backward()
        # Weight update
        optimizer.step()
    
    # Accuracy calculation
    output = (output > 0.5).float()
    accuracy = (output == label).float().mean()
    # Printing Statistics
    if (epoch+1)%10 == 0:
        print(f"Epoch: {epoch+1}/{epochs}, Loss: {loss:.3f}, Accuracy: {accuracy:.3f}")

Epoch: 10/200, Loss: 0.332, Accuracy: 0.906
Epoch: 20/200, Loss: 0.245, Accuracy: 0.875
Epoch: 30/200, Loss: 0.388, Accuracy: 0.812
Epoch: 40/200, Loss: 0.309, Accuracy: 0.906
Epoch: 50/200, Loss: 0.446, Accuracy: 0.781
Epoch: 60/200, Loss: 0.299, Accuracy: 0.938
Epoch: 70/200, Loss: 0.322, Accuracy: 0.906
Epoch: 80/200, Loss: 0.219, Accuracy: 0.906
Epoch: 90/200, Loss: 0.502, Accuracy: 0.719
Epoch: 100/200, Loss: 0.275, Accuracy: 0.875
Epoch: 110/200, Loss: 0.560, Accuracy: 0.781
Epoch: 120/200, Loss: 0.486, Accuracy: 0.781
Epoch: 130/200, Loss: 0.525, Accuracy: 0.719
Epoch: 140/200, Loss: 0.536, Accuracy: 0.656
Epoch: 150/200, Loss: 0.351, Accuracy: 0.938
Epoch: 160/200, Loss: 0.315, Accuracy: 0.844
Epoch: 170/200, Loss: 0.370, Accuracy: 0.812
Epoch: 180/200, Loss: 0.260, Accuracy: 0.938
Epoch: 190/200, Loss: 0.408, Accuracy: 0.750
Epoch: 200/200, Loss: 0.407, Accuracy: 0.719
